In [ ]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [ ]:
data = pd.read_excel("/content/1062_сабы.xlsx")
data

,Format:,Start,End,Style,text
0,Dialogue:,0:02:11.97,0:02:13.76,Санджи,Нога Ифрита!
1,Dialogue:,0:02:14.64,0:02:16.83,Default,Мужчины никогда не должны бить женщин!
2,Dialogue:,0:02:18.05,0:02:21.13,Default,Так было заведено\N ещё со времён динозавров!
3,Dialogue:,0:02:21.30,0:02:22.93,Default,Ты просто мразь!
4,Dialogue:,0:02:23.05,0:02:24.26,Санджи,Шея!
...,...,...,...,...,...
173,Dialogue:,0:23:26.95,0:23:31.16,Editors,Близится время когда пули их душ
174,Dialogue:,0:23:31.24,0:23:33.30,Editors,разрушат старую стену стоящую на пути
175,Dialogue:,0:23:33.32,0:23:35.40,Editors,их неуёмного стремления к будущему!
176,Dialogue:,0:23:36.38,0:23:41.09,Default,В следующем эпизоде {\i1}One Piece{\i0}: «Луфф...


In [ ]:
import sys
import requests
from pymystem3 import Mystem
m = Mystem()

def tag_mystem(
    text="Текст нужно передать функции в виде строки!", mapping=None, postags=True
):
    # если частеречные тэги не нужны (например, их нет в модели), выставьте postags=False
    # в этом случае на выход будут поданы только леммы

    processed = m.analyze(text)
    tagged = []
    for w in processed:
      if "analysis" in w and w["analysis"]:
        try:
            lemma = w["analysis"][0]["lex"].lower().strip()
            pos = w["analysis"][0]["gr"].split(",")[0]
            pos = pos.split("=")[0].strip()
            if mapping:
                if pos in mapping:
                    pos = mapping[pos]  # здесь мы конвертируем тэги
                else:
                    pos = "X"  # на случай, если попадется тэг, которого нет в маппинге
            tagged.append(lemma.lower() + "_" + pos)
        except KeyError:
            continue  # я здесь пропускаю знаки препинания, но вы можете поступить по-другому
    if not postags:
        tagged = [t.split("_")[0] for t in tagged]
    return tagged

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [ ]:
# Таблица преобразования частеречных тэгов Mystem в тэги UPoS:
mapping_url = "https://raw.githubusercontent.com/akutuzov/universal-pos-tags/4653e8a9154e93fe2f417c7fdb7a357b7d6ce333/ru-rnc.map"

mystem2upos = {}
r = requests.get(mapping_url, stream=True)
for pair in r.text.split("\n"):
    pair = pair.split()
    if len(pair) > 1:
        mystem2upos[pair[0]] = pair[1]


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))  # Вы можете указать нужный язык

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def text_prepare(text):
  filtered_word_list = []
  res = str(text).strip()
  list_of_normal_words = tag_mystem(text=res, mapping=mystem2upos)
  for word in list_of_normal_words:
    if word not in stop_words:
        filtered_word_list.append(word)
  return list_of_normal_words

In [ ]:
data["word_in_normal_view_with_tags"] = data["text"].progress_apply(text_prepare)

100%|██████████| 178/178 [00:00<00:00, 191.45it/s]


In [ ]:
def text_prepare_without_tags(text):
  filtered_word_list = []
  res = str(text).strip()
  list_of_normal_words = tag_mystem(text=res, mapping=mystem2upos, postags=False)
  for word in list_of_normal_words:
    if word not in stop_words:
        filtered_word_list.append(word)
  return filtered_word_list

In [ ]:
data["word_in_normal_view_without_tags_and_stopw"] = data["text"].progress_apply(text_prepare_without_tags)

100%|██████████| 178/178 [00:00<00:00, 2073.04it/s]


In [ ]:
data

,Format:,Start,End,Style,text,word_in_normal_view_with_tags,word_in_normal_view_without_tags_and_stopw
0,Dialogue:,0:02:11.97,0:02:13.76,Санджи,Нога Ифрита!,"[нога_NOUN, ифрит_NOUN]","[нога, ифрит]"
1,Dialogue:,0:02:14.64,0:02:16.83,Default,Мужчины никогда не должны бить женщин!,"[мужчина_NOUN, никогда_ADV, не_PART, должный_A...","[мужчина, должный, бить, женщина]"
2,Dialogue:,0:02:18.05,0:02:21.13,Default,Так было заведено\N ещё со времён динозавров!,"[так_ADV, быть_VERB, заводить_VERB, еще_ADV, с...","[заводить, время, динозавр]"
3,Dialogue:,0:02:21.30,0:02:22.93,Default,Ты просто мразь!,"[ты_PRON, просто_PART, мразь_NOUN]","[просто, мразь]"
4,Dialogue:,0:02:23.05,0:02:24.26,Санджи,Шея!,[шея_NOUN],[шея]
...,...,...,...,...,...,...,...
173,Dialogue:,0:23:26.95,0:23:31.16,Editors,Близится время когда пули их душ,"[близиться_VERB, время_NOUN, когда_SCONJ, пуля...","[близиться, время, пуля, душа]"
174,Dialogue:,0:23:31.24,0:23:33.30,Editors,разрушат старую стену стоящую на пути,"[разрушать_VERB, старый_ADJ, стена_NOUN, стоят...","[разрушать, старый, стена, стоять, путь]"
175,Dialogue:,0:23:33.32,0:23:35.40,Editors,их неуёмного стремления к будущему!,"[их_DET, неуемный_ADJ, стремление_NOUN, к_ADP,...","[неуемный, стремление, будущее]"
176,Dialogue:,0:23:36.38,0:23:41.09,Default,В следующем эпизоде {\i1}One Piece{\i0}: «Луфф...,"[в_ADP, следующий_ADJ, эпизод_NOUN, луффи_NOUN...","[следующий, эпизод, луффи, движение, момент, н..."


In [ ]:
data.to_excel("/content/1062_sab_prepared.xlsx")